In [ ]:
import os
import requests

from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown
from openai import OpenAI

In [ ]:
# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)

In [ ]:
# Load environment variables in a file called .env
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

In [ ]:
website = fetch_website_contents("https://www.allrecipes.com/easy-5-ingredient-dinners-11799627")
print(website)

In [ ]:
# Define System Prompt
system_prompt = """
You are a well-spoken cooking assistant that analyzes the contents of a food recipe website's meal list,
and finds a random recipe from within that list of recipes. You return the recipe name as a header, and if you cannot follow the link
use the internet to generate an appropriate recipe/dish summary or background in a well detailed paragraph.
Then you can return the list of ingredients followed by the cooking instructions for both conventional methods and air fryers 
(similarly, if you cannot follow the link, then use the internet to find or generate the appropriate ingredient list and instructions).
You do not need to select meals that include 'air fryer' in the name or already have air-fryer instructions. If a recipe does not have
air-fryer-related instructions, use the internet to find the best way to cook the recipe on an air fryer and generate the instructions.
Finally, display the information for that recipe. Format the ingredients (use this as a subheading) and instructions (use this as another subheading) 
into their own neatly organized tables.

You ignore text that might be website-navigation related that is not related to the link to the recipe.

You respond in markdown. Do not wrap the markdown in a code block!
"""

In [ ]:
# Define User Prompt

user_prompt_prefix = """
Here are the contents of a food recipe website's easy dinners list. Randomly choose one of the recipes. 
Return the recipe name, then provide a summary or background of the dish. Then return the ingredients, 
and cooking instructions for both conventional methods and air-frying.
"""

In [ ]:
openai = OpenAI()

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt_prefix + website}
]

response = openai.chat.completions.create(model="gpt-5-nano", messages=messages)
Markdown(response.choices[0].message.content)